In [ ]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

import geopandas as gpd
import pandas as pd
import folium 
import branca
import datetime as dt
from siuba import *
from bus_service_utils import better_bus_utils

from rt_analysis import rt_filter_map_plot

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

In [ ]:
import intake
catalog = intake.open_catalog('../bus_service_increase/catalog.yml')

In [ ]:
hwys = catalog.state_highway_network.read()

In [ ]:
hwys.shape

# 100 Recs - D11

## 1 - Corridor Improvements

In [ ]:
corridor_gdf_d11 = ((better_bus_utils.get_sorted_transit_routes(recommendation_category="corridor"))
                   >>filter(_.caltrans_district=="11 - San Diego")
                   >>head(10) #get top 10
                  )

In [ ]:
corridor_gdf_d11

In [ ]:
#corridor_gdf >> count(_.caltrans_district)

In [ ]:
corridor_gdf_d11 >> distinct(_.calitp_itp_id)

In [ ]:
corridor_d11 = (corridor_gdf_d11
               >> select(-_.service_date)
              )

corridor_d11.explore("route_id", legend=True, tiles="CartoDB positron")

In [ ]:
# for each transit operator, generate rtfiltermapper
sdmts = rt_filter_map_plot.from_gcs(itp_id=278, analysis_date=dt.date(2022,10,12))
sdmts.rt_trips >> head(5)

In [ ]:
# get small df of route_id and route_short_name from rt_trips
d11_routenames = (sdmts.rt_trips
                 >> count(_.route_id, _.route_short_name, _.route_long_name)
                 >> select(-_.n)
                )

d11_routenames >> head(5)

In [ ]:
# inner join w gdf of slow routes
corridor_d11_names = (corridor_d11
                     >> inner_join(_,d11_routenames)
                    )

corridor_d11_names

In [ ]:
# pass the route names as a filter parameter
routename_list_c = corridor_d11_names["route_short_name"].tolist()
routename_list_c

In [ ]:
sdmts.set_filter(start_time='06:00', end_time='09:00', route_names = routename_list_c)

In [ ]:
am = sdmts.segment_speed_map()

In [ ]:
am

In [ ]:
sdmts.set_filter(start_time='15:00', end_time='19:00', route_names = routename_list_c)

In [ ]:
sdmts.segment_speed_map()

In [ ]:
shape_id = '27_3_87'
stop_range = [16, 31]

In [ ]:
sdmts.autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
sdmts.set_filter(route_names=['27'])

In [ ]:
sdmts.corridor_metrics()

In [ ]:
sdmts.quick_map_corridor()

## Draft Corridor Recs, SDMTS, D11

* Entire Route 28 (nn mi, nn speed-based delay, nn schedule-based delay)
* Route 10, Goldfinch to 54 ()
    * paralleled by faster Rapid 215 E of Park
* Route 27, Mission to Moraga
* 

## 2 - Hotspot Improvements

In [ ]:
hotspot_gdf_d11 = ((better_bus_utils.get_sorted_transit_routes(recommendation_category="hotspot"))
                   >>filter(_.caltrans_district=="11 - San Diego")
                   >>head(10) #get top 10
                  )

In [ ]:
hotspot_gdf_d11

In [ ]:
hotspot_gdf_d11 = (hotspot_gdf_d11
               >> select(-_.service_date)
              )

hotspot_gdf_d11.explore("route_id", legend=True, tiles="CartoDB positron")

In [ ]:
# inner join w gdf of slow routes
hotspot_d8_names = (hotspot_gdf_d8
                     >> inner_join(_,D8_routenames)
                    )

hotspot_d8_names

In [ ]:
# pass the route names as a filter parameter
routename_list_hs = hotspot_d8_names["route_short_name"].tolist()
routename_list_hs

In [ ]:
D8_filtermapper.reset_filter()

In [ ]:
# corridor metrics for Route 24 (most linear route)
# constructed at geojson.io 
corridor_r24 = gpd.read_file('./corridors/d8_oldtown_temecula.geojson')
D8_filtermapper.add_corridor(corridor_r24)
D8_filtermapper.quick_map_corridor()

In [ ]:
D8_filtermapper.set_filter(route_names = routename_list_hs)

In [ ]:
# make maps etc - mapping with the hotspot didn't return both ways of traffic
m = D8_filtermapper.segment_speed_map(corridor=False)
m

In [ ]:
D8_filtermapper.corridor_metrics()

In [ ]:
D8_filtermapper.rt_trips >> filter(_.route_id=="24")